In [12]:
import pandas as pd
import re
import nltk
from nltk.tokenize import word_tokenize
import torch
from transformers import XLMRobertaTokenizer, XLMRobertaModel

In [2]:
# Ensure you have NLTK data files downloaded (for tokenization)
nltk.download('punkt')

[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\HP\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!


True

In [8]:

# Read the CSV file
df = pd.read_csv(r"C:\Users\HP\week 5\Amharic-Named-Entity-Recognition-\data\scraped_data.csv")

# Print the first few rows of the dataframe
print(df.head())


     sender                                            message  \
0  MerttEka  📣 🔠🔠🔠🔠🔠🔠🔠🔠🔠🔠\n\n📎 ይሄንን👉 t.me/MerttEka 👈 ተጭነው  ...   
1  MerttEka  📣 MiNI HUMIDIFIER\n\n✔️ የቤት ውስጥ መዓዛ ማጨሻ\n✔️ ቤት...   
2  MerttEka  📣  🔠🔠🔠🔠🔠🔠🔠  Silicone Baking Mould\n\n✔️ 12 የኩኪ...   
3  MerttEka                                                NaN   
4  MerttEka                                                NaN   

                   timestamp  \
0  2024-09-25 06:51:54+00:00   
1  2024-09-25 06:51:27+00:00   
2  2024-09-25 06:47:02+00:00   
3  2024-09-25 06:47:02+00:00   
4  2024-09-25 06:47:02+00:00   

                                               media  
0  MessageMediaDocument(nopremium=False, spoiler=...  
1  MessageMediaPhoto(spoiler=False, photo=Photo(i...  
2  MessageMediaPhoto(spoiler=False, photo=Photo(i...  
3  MessageMediaPhoto(spoiler=False, photo=Photo(i...  
4  MessageMediaPhoto(spoiler=False, photo=Photo(i...  


In [27]:
# Load the XLM-RoBERTa tokenizer and model
tokenizer = XLMRobertaTokenizer.from_pretrained('xlm-roberta-base')
model = XLMRobertaModel.from_pretrained('xlm-roberta-base')

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model = model.to(device)

# Tokenization using XLM-RoBERTa
def tokenize_and_embed(text):
    if isinstance(text, str):
        # Tokenize the text
        inputs = tokenizer(text, return_tensors="pt", padding=True, truncation=True)
        input_ids = inputs['input_ids'].to(device)
        
        # Generate embeddings using the model
        with torch.no_grad():
            outputs = model(input_ids)
        
        # The model returns multiple values, where 'last_hidden_state' contains the embeddings
        embeddings = outputs.last_hidden_state
        
        # Convert the embeddings to a NumPy array if you need
        embeddings_np = embeddings.cpu().numpy()
        return embeddings_np
    else:
        return None

# Apply tokenization and embedding generation to the dataframe
df['embeddings'] = df['message'].apply(tokenize_and_embed)

# The 'embeddings' column now contains the XLM-RoBERTa embeddings for each message


c:\Users\HP\week 5\Amharic-Named-Entity-Recognition-\.myenv\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
c:\Users\HP\week 5\Amharic-Named-Entity-Recognition-\.myenv\Lib\site-packages\huggingface_hub\file_download.py:147: UserWarning: `huggingface_hub` cache-system uses symlinks by default to efficiently store duplicated files but your machine does not support them in C:\Users\HP\.cache\huggingface\hub\models--xlm-roberta-base. Caching files will still work but in a degraded version that might require more space on your disk. This warning can be disabled by setting the `HF_HUB_DISABLE_SYMLINKS_WARNING` environment variable. For more details, see https://huggingface.co/docs/huggingface_hub/how-to-cache#limitations.
To support symlinks on Windows, you either need to activate Developer Mode or to run Python as an a

In [21]:
# Separate metadata from content
metadata = df[['sender', 'timestamp']]
content = df[['tokenized_message']]

In [22]:
# Store the preprocessed data
preprocessed_data = pd.concat([metadata, content], axis=1)

In [23]:
# Save the preprocessed data to a new CSV file
preprocessed_data.to_csv('preprocessed_data.csv', index=False)

In [ ]:
print(preprocessed_data.csv.head())

In [24]:
print("Preprocessing completed. The preprocessed data is saved as 'preprocessed_data.csv'.")

Preprocessing completed. The preprocessed data is saved as 'preprocessed_data.csv'.
